In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#!unzip '/content/drive/My Drive/실무가보이는머신러닝_딥러닝/chapter05/kfood/Testdata.zip' -d '/content/drive/My Drive/실무가보이는머신러닝_딥러닝/chapter05/kfood/Testdata/'

In [ ]:
# 음식 이미지 전처리
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from PIL import Image
import os, glob
import numpy as np

root_dir = "/content/drive/My Drive/실무가보이는머신러닝_딥러닝/chapter05/kfood/"

categories = ['Dolsotbab','Jeyugbokk-eum','Kimchi','Samgyeobsal','SoybeanPasteStew','chicken']

nb_classes = len(categories)

image_width = 64
image_height = 64
'''
X = []
Y = []

for idx, category in enumerate(categories) :
    image_dir = root_dir + category
    files = glob.glob(image_dir + "/" +"*.jpg") # 사용자가 제시한 조건에 맞는 파일명을 리스트 형식으로 반환 "*" <-와일드카드

    print(image_dir + "/" + "*.jpg")

    for i ,f in enumerate(files) :
        img = Image.open(f)
        img = img.convert("RGB")
        img = img.resize((image_width, image_height))
        data = np.asarray(img)
        X.append(data)
        Y.append(idx)
X = np.array(X)
Y = np.array(Y)
'''

In [ ]:
'''
X_train, X_test, Y_train, Y_test = train_test_split(X,Y)
xy = (X_train, X_test, Y_train, Y_test)
np.save(root_dir + "kfood.npy", xy)
'''

In [ ]:
import sys , os
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from keras.utils import np_utils
import numpy as np 

np_classes = len(categories)
image_size = 64

In [ ]:
import keras
print(keras.__version__)

In [ ]:
def load_dataset() : 
    np_load_old = np.load
    np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k) # numpy 버전문제 고쳐서 쓰면 오류 해결 allow_pickle = False -> True

    x_train, x_test, y_train , y_test = np.load(root_dir + "kfood.npy")
    x_train = x_train.astype("float") /256
    x_test = x_test.astype("float") /256
    y_train = np_utils.to_categorical(y_train, nb_classes)
    y_test = np_utils.to_categorical(y_test, nb_classes)
    return x_train, x_test , y_train, y_test


In [ ]:
def build_model(in_shape) : 
    model = Sequential()
    model.add(Convolution2D(32,(3,3),padding='same', input_shape=in_shape))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    model.add(Convolution2D(64,(3,3),padding='same'))
    model.add(Activation('relu'))
    model.add(Convolution2D(64,(3,3)))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(nb_classes))
    model.add(Activation('softmax'))
    model.compile(loss='binary_crossentropy',optimizer='rmsprop',metrics=['accuracy'])
    return model

In [ ]:
def model_train(x,y) : 
    model = build_model(x.shape[1:])
    print(x.shape[1:])
    model.fit(x,y,batch_size=32, epochs=30)

    return model

def model_eval(model, x, y ) : 
    score = model.evaluate(x,y)
    print('loss=', score[0])
    print('accuracy=', score[1])
    

In [ ]:
#np_load_old = np.load
#np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)
x_train, x_test, y_train, y_test = load_dataset()
print(x_train.shape)
'''
model = model_train(x_train, y_train)
model_eval(model, x_test , y_test)

model.save(root_dir+'kfood_model.h5')
'''

In [ ]:
x_train[1:].shape

In [ ]:

model = model_train(x_train, y_train)
#model_eval(model, x_test , y_test)

#model.save(root_dir+'kfood_model.h5')


In [ ]:
model_eval(model, x_test , y_test)
model.save(root_dir+'kfood_model.h5')